In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
class VaR_tasas:
            
    def __init__(self,ticker=None, interval='1d', date=None):
        
        self.ticker = ticker
        self.interval = interval
        try:
            data = yf.download(tickers=self.ticker,
                              interval=self.interval,
                              progress=False).Close
        except:
            print('Asegurate de agregar fecha de inicio, fin y ticker')
    
        self.data = pd.DataFrame(data)
        self.data['rendimientos'] = self.data.pct_change()
        self.data.dropna(inplace=True)
    
        self.prices = self.data.Close
        self.rendimientos = self.data.rendimientos
    
        
    def window_years(self, year=10, var=0.01):
        
        self.var = var
        self.year = year
        mount_data = int(252*self.year)
        
        self.amount = mount_data
        
        var_rolling = self.data[['rendimientos']].rolling(window=self.amount)\
        .apply(lambda df: df.quantile(self.var))
        
        var_rolling.dropna(inplace=True)
        
        self.var_rolling = var_rolling[var_rolling.index > '2020-01-01']
        
        self.rendimientos = self.data[['rendimientos']]\
        [self.data[['rendimientos']].index > '2020-01-01']
        
        self.outlier_var = self.rendimientos[self.rendimientos < self.var_rolling].dropna()
        self.index = self.rendimientos[self.rendimientos < self.var_rolling].dropna().index
        self.outlier_var_short = self.rendimientos[self.rendimientos > self.var_rolling].dropna()
        self.index_short = self.rendimientos[self.rendimientos > self.var_rolling].dropna().index
        
    def plot_window(self, outlier=True, outlier_short=False):
        plt.plot(self.rendimientos)
        plt.plot(self.var_rolling, label='VaR')
        if outlier:
            plt.scatter(self.index, self.outlier_var, color='red')
        elif outlier_short:
            plt.scatter(self.index_short, self.outlier_var_short, color='red')
        else:
            pass        

Cantidad de títulos. La terminación 3 o 10 hace referencia al bono de tres y diez años respectivamente.

In [3]:
titulos_3 = 30000
titulos_10 = 10000

Cargamos un `DataFrame` previo con datos diarios de las tasas de los respectivos bonos. Para este caso se utilizó una base previa; sin embargo, la clase permite trabajar con datos de `yfinance`.

In [4]:
df = pd.read_excel('historico_tasas.xlsx').set_index('Date')
df.head()

,tasa3,tasa10
Date,,
06/09/2022,9.32,9.05
07/09/2022,9.31,9.03
08/09/2022,9.26,8.99
09/09/2022,9.24,8.90
12/09/2022,9.19,8.91


`pb_change` hace referencia a los cambios en los puntos base de la tasa de cada bono, siendo un punto base equivalente a 0.01

In [8]:
df['pb_change3'] = df.tasa3.diff()*100
df['pb_change10'] = df.tasa10.diff()*100

In [10]:
df.head()

,tasa3,tasa10,pb_change3,pb_change10
Date,,,,
06/09/2022,9.32,9.05,NaN,NaN
07/09/2022,9.31,9.03,-1.0,-2.0
08/09/2022,9.26,8.99,-5.0,-4.0
09/09/2022,9.24,8.90,-2.0,-9.0
12/09/2022,9.19,8.91,-5.0,1.0


`dv01` es el cambio en el precio del bono ante un cambio de un punto base en la tasa de interés.

In [11]:
dv01_3 = 0.024
dv01_10 = 0.063

Calculamos posteriormente el _Profit and Loss_ (P&L). En el caso del bono a 10 años omitimos el `-1`; sin embargo, puede plantearse expresando los títulos en negativo y dejar la fórmula igual que en el caso del Largo.

In [13]:
df['p_l_3'] = titulos_3 * dv01_3 * df.pb_change3 * -1
df['p_l_10'] = titulos_10 * dv01_10 * df.pb_change10

In [14]:
df

,tasa3,tasa10,pb_change3,pb_change10,p_l_3,p_l_10
Date,,,,,,
06/09/2022,9.32,9.05,NaN,NaN,NaN,NaN
07/09/2022,9.31,9.03,-1.0,-2.0,720.0,-1260.0
08/09/2022,9.26,8.99,-5.0,-4.0,3600.0,-2520.0
09/09/2022,9.24,8.90,-2.0,-9.0,1440.0,-5670.0
12/09/2022,9.19,8.91,-5.0,1.0,3600.0,630.0
...,...,...,...,...,...,...
25/08/2023,9.98,9.24,3.0,1.0,-2160.0,630.0
28/08/2023,10.00,9.24,2.0,0.0,-1440.0,0.0
29/08/2023,9.97,9.22,-3.0,-2.0,2160.0,-1260.0


Para calcular el P&L del portafolio sumamos ambos P&L correspondientes a los bonos.

In [16]:
df['p_l_port'] = df.p_l_3 + df.p_l_10

In [17]:
df

,tasa3,tasa10,pb_change3,pb_change10,p_l_3,p_l_10,p_l_port
Date,,,,,,,
06/09/2022,9.32,9.05,NaN,NaN,NaN,NaN,NaN
07/09/2022,9.31,9.03,-1.0,-2.0,720.0,-1260.0,-540.0
08/09/2022,9.26,8.99,-5.0,-4.0,3600.0,-2520.0,1080.0
09/09/2022,9.24,8.90,-2.0,-9.0,1440.0,-5670.0,-4230.0
12/09/2022,9.19,8.91,-5.0,1.0,3600.0,630.0,4230.0
...,...,...,...,...,...,...,...
25/08/2023,9.98,9.24,3.0,1.0,-2160.0,630.0,-1530.0
28/08/2023,10.00,9.24,2.0,0.0,-1440.0,0.0,-1440.0
29/08/2023,9.97,9.22,-3.0,-2.0,2160.0,-1260.0,900.0


El VaR del portafolio a un nivel de confianza del 97.5%:

In [24]:
var_port = df[['p_l_port']].quantile(0.025)
var_port

p_l_port   -5967.0
Name: 0.025, dtype: float64

CVaR o ES:

In [34]:
cvar_port = df[['p_l_port']][df.p_l_port < var_port[0]].mean()
cvar_port

p_l_port   -9347.142857
dtype: float64